In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [53]:
# 网络结构

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 卷积层 '1'表示输入图片为单通道, '6'表示输出通道数, '5'表示卷积核为5*5
        # 核心
        # 初始化的过程中其实没有再定义网络结构，只是定义了一些函数
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 仿射层/全连接层: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        # 这里的前向过程才定义了整个网络结构
    def forward(self, x):
        # 在由多个输入平面组成的输入信号上应用2D最大池化.
        # (2, 2) 代表的是池化操作的步幅
        
        # 这里正是从输入层，通过一个卷积之后，在经过一个pool
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 如果大小是正方形, 则只能指定一个数字
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        # 这边便是将x拉值，以便用于全连接
        x = x.view(-1, self.num_flat_features(x))
        # 接下来就是普通的两个全连接层
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # 下面是输出层
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # 除批量维度外的所有维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [20]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [21]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [22]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

tensor([[ 0.1560,  0.0531,  0.0260,  0.1314, -0.0677, -0.0443,  0.1241,
          0.0471,  0.0629,  0.0972]])


In [12]:
net.zero_grad()
out.backward(torch.randn(1, 10))

### 损失函数

In [55]:
output = net(input)
output = output.view(-1)
print(output)
target = Variable(torch.arange(1, 11))  # 一个虚拟的目标
print(target)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor([ 0.2415,  0.1231,  0.0212,  0.2614, -0.0003,  0.0659,  0.3496,
         0.1544,  0.2534,  0.3616])
tensor([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.])
tensor(36.2347)


### 反向传播

In [56]:
net.zero_grad()     # 把之前的梯度清零

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([ 0.,  0.,  0.,  0.,  0.,  0.])
conv1.bias.grad after backward
tensor([ 0.0121, -0.1233, -0.4033, -0.0280, -0.2666, -0.3001])


### 更新权重

In [47]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [57]:
import torch.optim as optim

# 新建一个优化器, 指定要调整的参数和学习率
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# 在训练过程中:
optimizer.zero_grad()   # 首先梯度清零(与 net.zero_grad() 效果一样)
output = net(input)
output = output.view(-1)

loss = criterion(output, target)

loss.backward()
optimizer.step()    # 更新参数